In [22]:
import yfinance as yf
import pandas as pd
import numpy as np

tickers = ["SPY", "QQQ", "AAPL", "MSFT"]
interval = "1m"
period = "5d"
initial_investment = 10000
backup_cash = 10000
starting_capital = initial_investment + backup_cash
Z_BUY = -1.0
Z_SELL = 1.0
MAX_POSITION_RATIO = 0.75
MIN_CASH_BUFFER = 2000
window = 20

results = []

for symbol in tickers:
    print(f"Running: {symbol}")
    try:
        df = yf.download(symbol, interval=interval, period=period)
        df.columns = df.columns.get_level_values(0)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()

        if df.index.tz is None:
            df.index = df.index.tz_localize("America/New_York")
        else:
            df.index = df.index.tz_convert("America/New_York")

        df = df.between_time("09:30", "16:00")
        df = df[df.index.dayofweek < 5]
        df['Close'] = df['Close'].astype(float)
        df['zscore'] = (df['Close'] - df['Close'].rolling(window).mean()) / df['Close'].rolling(window).std()

        shares = 0
        buy_count = 0
        sell_count = 0
        has_liquidated = False
        backup_cash_copy = backup_cash

        first_price = df['Close'].iloc[window]
        shares = initial_investment / first_price
        entry_price = first_price

        buy_count += 1

        for i in range(window + 1, len(df)):
            price = df['Close'].iloc[i]
            z = df['zscore'].iloc[i]
            equity_value = shares * price
            portfolio_value = backup_cash_copy + equity_value

            if has_liquidated:
                continue

            position_value = shares * price
            max_position_value = portfolio_value * MAX_POSITION_RATIO

            if z < Z_BUY:
                drop_pct = abs(z) / 3
                investment = min(backup_cash_copy * drop_pct, max_position_value - position_value)
                if backup_cash_copy - investment >= MIN_CASH_BUFFER and investment > 0:
                    bought_shares = investment / price
                    shares += bought_shares
                    backup_cash_copy -= investment
                    buy_count += 1

            elif z > Z_SELL and shares > 0:
                climb_pct = z / 3
                sell_shares = min(shares, shares * climb_pct)
                if sell_shares > 0:
                    proceeds = sell_shares * price
                    backup_cash_copy += proceeds
                    shares -= sell_shares
                    sell_count += 1

        final_price = df['Close'].iloc[-1]
        final_value = backup_cash_copy + shares * final_price
        cumulative_return = (final_value - starting_capital) / starting_capital * 100

        bh_shares = initial_investment / entry_price
        bh_value = bh_shares * final_price
        bh_return = (bh_value - initial_investment) / initial_investment * 100

        results.append({
            "Ticker": symbol,
            "Final Value": round(final_value, 2),
            "Return (%)": round(cumulative_return, 2),
            "Buy & Hold Return (%)": round(bh_return, 2),
            "Outperformed/Underperfromed (%)": round(cumulative_return-bh_return,2),
            "Buys": buy_count,
            "Sells": sell_count,
            "Trades": buy_count + sell_count
        })

    except Exception as e:
        print(f"Error for {symbol}: {e}")

# Final results
results_df = pd.DataFrame(results)
print("\nMulti-Ticker Strategy Results:\n")
display(results_df)


Running: SPY


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Running: QQQ
Running: AAPL


[*********************100%***********************]  1 of 1 completed


Running: MSFT


[*********************100%***********************]  1 of 1 completed


Multi-Ticker Strategy Results:



,Ticker,Final Value,Return (%),Buy & Hold Return (%),Outperformed/Underperfromed (%),Buys,Sells,Trades
0,SPY,19693.10,-1.53,-2.83,1.30,335,486,821
1,QQQ,19724.60,-1.38,-4.41,3.03,334,495,829
2,AAPL,20406.47,2.03,-1.23,3.26,276,488,764
3,MSFT,19663.81,-1.68,-3.38,1.70,334,480,814
